**Name: Aakash Thatte                   
Roll number: 12041590**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
import os
import random
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import imageio
import pickle

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Semester4/DS250/test_images_paths.csv")
testing_hogs = pd.read_csv("/content/drive/MyDrive/Semester4/DS250/hog_256_test.csv")
hog_averages = pd.read_csv("/content/drive/MyDrive/Semester4/DS250/hog_256_averages.csv")
testing_hogs.drop("Unnamed: 0",axis=1,inplace=True)
test.drop("Unnamed: 0",axis=1,inplace=True)
hog_averages.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
hogs = {}
for c in hog_averages.columns:
  hogs[c] = hog_averages[c].values

avg_hogs = list(hogs.values())

In [ ]:
fruit_class_testing_folder = "/content/drive/MyDrive/Semester4/DS250/Training/Training_actual/Test/"
fruit_class_training_folder = "/content/drive/MyDrive/Semester4/DS250/Training/Training_actual/Training/"
all_fruit_classes = os.listdir(fruit_class_testing_folder)

def process_name(name):
  name = name.lower()
  names = name.split()
  return ("_").join(names)

In [ ]:
with open("/content/drive/MyDrive/Semester4/DS250/class_rgb_average.pkl","rb") as f:
    clavg = pickle.load(f)
def give_r_g_b_average(image):
  return (image[:,:,0].sum()/10000,image[:,:,1].sum()/10000,image[:,:,2].sum()/10000)

averages_rgb = list(clavg.values())

In [ ]:
np.array(averages_rgb).shape

(131, 3)

In [ ]:
import scipy

In [ ]:
def get_k(x,X,k,**kwargs):
    """
    finds the indices of k-nearest neighbors of x in X
    """
    d = scipy.spatial.distance.cdist(x.reshape(1,-1), X, **kwargs).flatten()
    return np.argpartition(d, k)[:k]

In [ ]:
testing_img_label_df = pd.read_csv("/content/drive/MyDrive/Semester4/DS250/testing_image_label.csv")
training_img_label_df = pd.read_csv("/content/drive/MyDrive/Semester4/DS250/training_image_label.csv")

In [ ]:
# test_rgb = []
# for i,image_name in enumerate(testing_hogs.columns):
#   #print(testing_img_label_df.iloc[i]['image'],image_name)
#   test_rgb.append(give_r_g_b_average(imread(fruit_class_testing_folder+test.iloc[i]['image'])))

In [ ]:
#avg_hogs

In [ ]:
from collections import Counter
pred = []
labels = []
def combine():
  hog = None
  rgb = None
  #print(len(np.array(avg_hogs)))
  for i,c in enumerate(testing_hogs.columns):
    x = testing_hogs[c].values
    arr = get_k(np.array(x),np.array(avg_hogs),k=3)
    hog = [training_img_label_df.iloc[i]['label'] for i in arr]
    #print(hog)
    
    xx = give_r_g_b_average(imread(fruit_class_testing_folder+test.iloc[i]['image']))
    ar = get_k(np.array(xx),np.array(averages_rgb),k=3)
    rgb = [training_img_label_df.iloc[i]['label'] for i in ar]
    #print(rgb)

    all = hog.copy()
    all.extend(rgb)
    #print(all)
    pred.append(Counter(all).most_common(1)[0][0])
    labels.append(test[test['id']==c]['label'])

    if (i+1)%500==0:
      print(i+1,"images processed")

In [ ]:
combine()

500 images processed
1000 images processed
1500 images processed
2000 images processed
2500 images processed
3000 images processed
3500 images processed
4000 images processed
4500 images processed
5000 images processed
5500 images processed
6000 images processed
6500 images processed
7000 images processed
7500 images processed
8000 images processed
8500 images processed
9000 images processed
9500 images processed
10000 images processed
10500 images processed
11000 images processed
11500 images processed
12000 images processed
12500 images processed
13000 images processed
13500 images processed
14000 images processed
14500 images processed
15000 images processed
15500 images processed
16000 images processed
16500 images processed
17000 images processed
17500 images processed
18000 images processed
18500 images processed
19000 images processed
19500 images processed
20000 images processed
20500 images processed
21000 images processed
21500 images processed
22000 images processed
22500 im

In [ ]:
combined_frame = pd.DataFrame(columns=['label','prediction'])
combined_frame['label'] = labels
combined_frame['prediction'] = pred

In [ ]:
combined_frame['actual'] = combined_frame['label'].apply(lambda x:x.values[0])

In [ ]:
len(combined_frame[combined_frame['actual']==combined_frame['prediction']]),len(combined_frame)

(157, 22688)

In [ ]:
# As we can see, the combined model is giving ever worse accuracy, exactly opposite to what was expected.

In [ ]:
combined_frame

,label,prediction,actual
0,"9678 kohlrabi Name: label, dtype: object",kohlrabi,kohlrabi
1,"9621 kohlrabi Name: label, dtype: object",kohlrabi,kohlrabi
2,"9717 kohlrabi Name: label, dtype: object",kohlrabi,kohlrabi
3,"9769 kohlrabi Name: label, dtype: object",kohlrabi,kohlrabi
4,"9758 kohlrabi Name: label, dtype: object",kohlrabi,kohlrabi
...,...,...,...
22683,"9460 kaki Name: label, dtype: object",kohlrabi,kaki
22684,"9440 kaki Name: label, dtype: object",kohlrabi,kaki
22685,"9413 kaki Name: label, dtype: object",kohlrabi,kaki
22686,"9298 kaki Name: label, dtype: object",kohlrabi,kaki


In [ ]:
test

,image,label,id
0,Apple Braeburn/3_100.jpg,apple_braeburn,apple_braeburn_3_100.jpg
1,Apple Braeburn/32_100.jpg,apple_braeburn,apple_braeburn_32_100.jpg
2,Apple Braeburn/321_100.jpg,apple_braeburn,apple_braeburn_321_100.jpg
3,Apple Braeburn/322_100.jpg,apple_braeburn,apple_braeburn_322_100.jpg
4,Apple Braeburn/323_100.jpg,apple_braeburn,apple_braeburn_323_100.jpg
...,...,...,...
22683,Watermelon/r_95_100.jpg,watermelon,watermelon_r_95_100.jpg
22684,Watermelon/r_96_100.jpg,watermelon,watermelon_r_96_100.jpg
22685,Watermelon/r_97_100.jpg,watermelon,watermelon_r_97_100.jpg
22686,Watermelon/r_98_100.jpg,watermelon,watermelon_r_98_100.jpg


In [ ]:
training_img_label_df

,Unnamed: 0,image,label
0,0,kohlrabi_r_86_100.jpg,kohlrabi
1,1,kohlrabi_r_185_100.jpg,kohlrabi
2,2,kohlrabi_182_100.jpg,kohlrabi
3,3,kohlrabi_r_226_100.jpg,kohlrabi
4,4,kohlrabi_r_53_100.jpg,kohlrabi
...,...,...,...
67687,67687,kaki_117_100.jpg,kaki
67688,67688,kaki_181_100.jpg,kaki
67689,67689,kaki_r_305_100.jpg,kaki
67690,67690,kaki_41_100.jpg,kaki


In [ ]:
testing_img_label_df

,Unnamed: 0,image,label
0,0,kohlrabi_32_100.jpg,kohlrabi
1,1,kohlrabi_15_100.jpg,kohlrabi
2,2,kohlrabi_r_137_100.jpg,kohlrabi
3,3,kohlrabi_r_302_100.jpg,kohlrabi
4,4,kohlrabi_r_292_100.jpg,kohlrabi
...,...,...,...
22683,22683,kaki_r_85_100.jpg,kaki
22684,22684,kaki_r_193_100.jpg,kaki
22685,22685,kaki_r_166_100.jpg,kaki
22686,22686,kaki_0_100.jpg,kaki
